In [2]:
import os
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
from datatable import dt, fread
# With python v>3.9 need to install datatable from source for the time being
# pip install git+https://github.com/h2oai/datatable

In [10]:
#annot = pd.read_csv("PRJNA383416_extracted_condition_genotype_selected.csv")

# IBM RNA-seq samples
annot = pd.read_csv("PRJNA123456_edited.csv")
annot.rename(columns = {'run_alias':'genotype'}, inplace = True)

annot.head()

,run_accession,sample_accession,study_accession,tax_id,scientific_name,instrument_model,library_strategy,read_count,genotype,sample_alias,fastq_ftp,fastq_md5
0,SRR516201,SAMN01057777,PRJNA179160,4577,Zea mays,Illumina HiSeq 2000,RNA-Seq,28337804,M0001,Maize_IBM_RIL_Shoot_Apex_M0001,ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/SRR516201/...,5b51e0dd31109b0c1e97a511ef3f0450
1,SRR516795,SAMN01057778,PRJNA179160,4577,Zea mays,Illumina HiSeq 2000,RNA-Seq,32660449,M0005,Maize_IBM_RIL_Shoot_Apex_M0005,ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/SRR516795/...,7cc6577cee718b4905ef1c2496b88314
2,SRR516796,SAMN01083909,PRJNA179160,4577,Zea mays,Illumina HiSeq 2000,RNA-Seq,34673395,M0007,Maize_IBM_RIL_Shoot_Apex_M0007,ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/SRR516796/...,c7fa8d8c20ab22ea8f7f382adc9fea5c
3,SRR516797,SAMN01083907,PRJNA179160,4577,Zea mays,Illumina HiSeq 2000,RNA-Seq,43740590,M0008,Maize_IBM_RIL_Shoot_Apex_M0008,ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/SRR516797/...,3f2fcf01661faaed79e51accce47ef8b
4,SRR516798,SAMN01083910,PRJNA179160,4577,Zea mays,Illumina HiSeq 2000,RNA-Seq,31515426,M0010,Maize_IBM_RIL_Shoot_Apex_M0010,ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/SRR516798/...,417b590b4a72807d20c3ad3158713515


In [13]:
try:
    os.mkdir("ramdisk/geno_kmers")
except:
    None

genotypes = set(annot["genotype"].values)
genotype_counter = len(genotypes)
print("Total genotypes:", genotype_counter)
for geno in genotypes:
    acc = annot[annot["genotype"] == geno]["run_accession"].values.tolist()
    if len(acc)==1:
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(acc[0]))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(acc[0]))
        os.system('pigz --decompress ramdisk/*')
        os.system('mv ramdisk/{0}.kmc_suf ramdisk/geno_kmers/{1}.kmc_suf'.format(acc[0], geno))
        os.system('mv ramdisk/{0}.kmc_pre ramdisk/geno_kmers/{1}.kmc_pre'.format(acc[0], geno))
    if len(acc)>=2:
        fl1, fl2 = acc[0], acc[1]
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl1))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl1))
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl2))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl2))
        os.system('pigz --decompress ramdisk/*')
        os.system('kmc_tools simple ramdisk/{0} ramdisk/{1} union ramdisk/kmer_union0'.format(fl1, fl2))
        os.system('rm {0}/{1}.kmc_suf {0}/{1}.kmc_pre {0}/{2}.kmc_suf {0}/{2}.kmc_pre'.format("ramdisk", fl1, fl2))
    if len(acc)==2:
        os.system('mv ramdisk/kmer_union0.kmc_suf ramdisk/geno_kmers/{0}.kmc_suf'.format(geno))
        os.system('mv ramdisk/kmer_union0.kmc_pre ramdisk/geno_kmers/{0}.kmc_pre'.format(geno))
    if len(acc)>2:
        acc = acc[2:]
        # kmc_tools can't overwrite a file so have to iterate back and 
        # forth with the output file to union merge multiple kmer files
        # This also seems to be an easier solution than the internal 'complex'
        count = 1
        for fl in acc:
            os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl))
            os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl))
            os.system('pigz --decompress ramdisk/*')
            prv, nxt = str((count-1)%2), str(count%2)
            os.system('kmc_tools simple ramdisk/kmer_union{0} ramdisk/{1} union ramdisk/kmer_union{2}'.format(prv, fl, nxt))
            os.system('rm {0}/{1}.kmc_suf {0}/{1}.kmc_pre {0}/kmer_union{2}.kmc_suf {0}/kmer_union{2}.kmc_pre'.format("ramdisk", fl, prv))
            count+=1
        # Done union merge of group kmers. Read table 
        os.system('mv ramdisk/kmer_union{0}.kmc_suf ramdisk/geno_kmers/{1}.kmc_suf'.format(nxt, geno))
        os.system('mv ramdisk/kmer_union{0}.kmc_pre ramdisk/geno_kmers/{1}.kmc_pre'.format(nxt, geno))
    genotype_counter = genotype_counter - 1
    print("Genotypes remaining:", genotype_counter)


Total genotypes: 107


pigz: skipping: ramdisk/geno_kmers is a directory


Genotypes remaining: 106


pigz: skipping: ramdisk/geno_kmers is a directory


Genotypes remaining: 105


...

pigz: skipping: ramdisk/geno_kmers is a directory


Genotypes remaining: 2


pigz: skipping: ramdisk/geno_kmers is a directory


Genotypes remaining: 1
Genotypes remaining: 0


pigz: skipping: ramdisk/geno_kmers is a directory


# Get the combined kmer file for all genotypes 

In [14]:
genotypes = list(set(annot["genotype"].values))
genotype_counter = len(genotypes)
print("Total genotypes:", genotype_counter)

fl1, fl2 = genotypes[0], genotypes[1]
os.system('kmc_tools simple ramdisk/geno_kmers/{0} ramdisk/geno_kmers/{1} union ramdisk/kmer_union0'.format(fl1, fl2))

genotype_counter = genotype_counter - 2
print("Total genotypes:", genotype_counter)

count = 1
for geno in genotypes[2:]:
    # kmc_tools can't overwrite a file so have to iterate back and 
    # forth with the output file to union merge multiple kmer files
    # This also seems to be an easier solution than the internal 'complex'
    prv, nxt = str((count-1)%2), str(count%2)
    os.system('kmc_tools simple ramdisk/kmer_union{0} ramdisk/geno_kmers/{1} union ramdisk/kmer_union{2}'.format(prv, geno, nxt))
    os.system('rm {0}{1}.kmc_suf {0}{1}.kmc_pre'.format("ramdisk/kmer_union", prv))
    count+=1

    genotype_counter = genotype_counter - 1
    print("Genotypes remaining:", genotype_counter)

# Rename the file
os.system('mv {0}{1}.kmc_suf {0}_all.kmc_suf'.format("ramdisk/kmer_union", nxt))
os.system('mv {0}{1}.kmc_pre {0}_all.kmc_pre'.format("ramdisk/kmer_union", nxt))

Total genotypes: 107


in1: 100% in2: 100% 


Total genotypes: 105


in1: 100% in2: 100% 


Genotypes remaining: 104


...

in1: 100% in2: 100% 


Genotypes remaining: 1


in1: 100% in2: 100% 


Genotypes remaining: 0


0

# Generate the kmer count files for each genotype

In [57]:
genotypes = set(annot["genotype"].values)
genotype_counter = len(genotypes)
print("Total genotypes:", genotype_counter)

count = 0
for geno in genotypes:
    # I wanted to use -cs1 (-cs<value>  - maximal value of a counter) but it doesn't seem to work
    os.system('kmc_tools simple ramdisk/kmer_union_all ramdisk/geno_kmers/{0} union ramdisk/tmp -ocright'.format(geno))
    os.system('kmc_dump ramdisk/tmp ramdisk/tmp_dump')
    if count == 0:
        dfk = dt.fread("ramdisk/tmp_dump", sep="\t")
        ixs = dfk["C0"].to_pandas()["C0"].values.tolist()
        dfk = dfk["C1"]
        count = -1
        continue
    dfk = dt.cbind(dfk, dt.fread("ramdisk/tmp_dump", sep="\t")["C0"])

Total genotypes: 107


in1: 100% in2: 100% 
in1: 100% in2: 100% 
in1: 100% in2: 100% 
in1: 100% in2: 100% 


IOError: Too few fields on line 96328990: expected 2 but found only 1 (with sep='t'). Set fill=True to ignore this error.  <<AGCAGCAGGGTATGGATGTTGGGGG>>

# Combine k-mers to Hapmap file

In [55]:
t1 = dt.fread("ramdisk/tmp_dump", sep="\t", max_nrows=100)["C1"]
t1 = dt.cbind(t1, t1)
t1 = dt.cbind(t1, t1)
t1

,C1,C2,C3,C4
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,255,255,255,255
1,113,113,113,113
2,115,115,115,115
3,35,35,35,35
4,69,69,69,69
5,48,48,48,48
6,10,10,10,10
7,20,20,20,20
8,55,55,55,55


In [36]:
?dt.fread

In [50]:
dfk = dt.fread("ramdisk/tmp_dump", sep="\t", max_nrows=100)
ixs = dfk["C0"].to_pandas()["C0"].values.tolist()

['AAAAAAAAAAAAAAAAAAAAAAAAA',
 'AAAAAAAAAAAAAAAAAAAAAAAAC',
 'AAAAAAAAAAAAAAAAAAAAAAAAG',
 'AAAAAAAAAAAAAAAAAAAAAAAAT',
 'AAAAAAAAAAAAAAAAAAAAAAACA',
 'AAAAAAAAAAAAAAAAAAAAAAACC',
 'AAAAAAAAAAAAAAAAAAAAAAACG',
 'AAAAAAAAAAAAAAAAAAAAAAACT',
 'AAAAAAAAAAAAAAAAAAAAAAAGA',
 'AAAAAAAAAAAAAAAAAAAAAAAGC',
 'AAAAAAAAAAAAAAAAAAAAAAAGG',
 'AAAAAAAAAAAAAAAAAAAAAAAGT',
 'AAAAAAAAAAAAAAAAAAAAAAATA',
 'AAAAAAAAAAAAAAAAAAAAAAATC',
 'AAAAAAAAAAAAAAAAAAAAAAATG',
 'AAAAAAAAAAAAAAAAAAAAAAATT',
 'AAAAAAAAAAAAAAAAAAAAAACAA',
 'AAAAAAAAAAAAAAAAAAAAAACAC',
 'AAAAAAAAAAAAAAAAAAAAAACAG',
 'AAAAAAAAAAAAAAAAAAAAAACAT',
 'AAAAAAAAAAAAAAAAAAAAAACCA',
 'AAAAAAAAAAAAAAAAAAAAAACCC',
 'AAAAAAAAAAAAAAAAAAAAAACCG',
 'AAAAAAAAAAAAAAAAAAAAAACCT',
 'AAAAAAAAAAAAAAAAAAAAAACGA',
 'AAAAAAAAAAAAAAAAAAAAAACGC',
 'AAAAAAAAAAAAAAAAAAAAAACGG',
 'AAAAAAAAAAAAAAAAAAAAAACGT',
 'AAAAAAAAAAAAAAAAAAAAAACTA',
 'AAAAAAAAAAAAAAAAAAAAAACTC',
 'AAAAAAAAAAAAAAAAAAAAAACTG',
 'AAAAAAAAAAAAAAAAAAAAAACTT',
 'AAAAAAAAAAAAAAAAAAAAAAGAA',
 'AAAAAAAA